In [22]:
from pathlib import Path

import pandas as pd

from gtfs_tools.gtfs import GtfsDataset
from gtfs_tools.utils.gtfs import add_modality_descriptions

In [7]:
gtfs_parent_dir = Path(r'D:\projects\gtfs-tools\data\raw\gtfs_la_cbsa')

In [8]:
gtfs_pth_lst = [pth.parent for pth in gtfs_parent_dir.glob('**/agency.txt')]

In [24]:
for gtfs_pth in gtfs_pth_lst:

    # create a gtfs dataset 
    gtfs = GtfsDataset(gtfs_pth, standardize_route_types=True)

    # get the needed columns for calculating overnight service
    tm_df = gtfs.stop_times.data.loc[:,['stop_id', 'arrival_time']]

    # calculate hour of the day by getting the timedelta hours and retaining the remainder when dividing by 24
    tm_df['hours'] = tm_df['arrival_time'].dt.components.hours % 24

    # calculate late night by evaluating if the stop time is after an hour in the evening or before an hour in the morning
    tm_df['late_night'] = (tm_df['hours'] <= 3) | (tm_df['hours'] >= 23)

    # get just the overnight boolean
    late_df = tm_df.loc[:,['stop_id', 'late_night']].set_index('stop_id')

    # get data frame of stops, routes and route types
    rt_df = pd.merge(gtfs._crosstab_stop_route, gtfs.routes.data.loc[:,['route_id', 'route_type']], on='route_id', how='left')

    # flag fixed types (not bus or school bus)
    rt_df['fixed_typ'] = ~rt_df['route_type'].isin([3, 31])

    # add factor for each route; fixed routes are weighted 3x more than bus routes
    rt_df['weighted_route_factor'] = rt_df['fixed_typ'].apply(lambda val: 3 if val else 1)

    # sum weighted route factor by stop
    stop_rt_df = rt_df[['stop_id', 'route_type', 'weighted_route_factor']].groupby('stop_id').sum()

    # add the properties onto a spatailly enabled data frame
    stops_sedf = (
        gtfs.stops.sedf
        .join(gtfs.stops.trip_count, on='stop_id', how='left')
        .join(late_df, on='stop_id', how='left')
        .join(stop_rt_df, on='stop_id', how='left')
    )
    
    break
    
stops_sedf

,stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,level_id,platform_code,SHAPE,trip_count,late_night,route_type,weighted_route_factor
0,0a40ca18-a300-492f-b261-9c7c7278d617,NaN,Katella-Auburn,NaN,NaN,33.803473,-117.893838,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.893837668295, ""y"": 33.8034729182036...",25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,NaN,Katella-Auburn,NaN,NaN,33.803473,-117.893838,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.893837668295, ""y"": 33.8034729182036...",25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,NaN,Katella-Auburn,NaN,NaN,33.803473,-117.893838,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.893837668295, ""y"": 33.8034729182036...",25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,NaN,Katella-Auburn,NaN,NaN,33.803473,-117.893838,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.893837668295, ""y"": 33.8034729182036...",25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,NaN,Katella-Auburn,NaN,NaN,33.803473,-117.893838,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.893837668295, ""y"": 33.8034729182036...",25,False,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,19,NaN,Disneyland Main Transportation Center - 19,NaN,Canyon Line,33.809569,-117.916393,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.916393, ""y"": 33.809569, ""spatialRef...",7,False,33,6
85,19,NaN,Disneyland Main Transportation Center - 19,NaN,Canyon Line,33.809569,-117.916393,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.916393, ""y"": 33.809569, ""spatialRef...",7,False,33,6
85,19,NaN,Disneyland Main Transportation Center - 19,NaN,Canyon Line,33.809569,-117.916393,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.916393, ""y"": 33.809569, ""spatialRef...",7,False,33,6
85,19,NaN,Disneyland Main Transportation Center - 19,NaN,Canyon Line,33.809569,-117.916393,NaN,NaN,0,NaN,NaN,0,NaN,NaN,"{""x"": -117.916393, ""y"": 33.809569, ""spatialRef...",7,False,33,6


In [23]:
stops_sedf = stops_sedf.loc[:,['stop_id', 'stop_name', 'trip_count', 'late_night', 'route_type', 'weighted_route_factor', 'SHAPE']]

stops_sedf

,stop_id,stop_name,trip_count,late_night,route_type,weighted_route_factor
0,0a40ca18-a300-492f-b261-9c7c7278d617,Katella-Auburn,25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,Katella-Auburn,25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,Katella-Auburn,25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,Katella-Auburn,25,False,3,3
0,0a40ca18-a300-492f-b261-9c7c7278d617,Katella-Auburn,25,False,3,3
...,...,...,...,...,...,...
85,19,Disneyland Main Transportation Center - 19,7,False,33,6
85,19,Disneyland Main Transportation Center - 19,7,False,33,6
85,19,Disneyland Main Transportation Center - 19,7,False,33,6
85,19,Disneyland Main Transportation Center - 19,7,False,33,6


In [20]:
rt_df

,stop_id,route_id,route_type,fixed_typ,weighted_route_factor
0,9,fe28ed1e-274e-402a-b0c1-3f8d1bf9b7c5,3,True,3
1,8e75ec44-01e5-4319-adbe-d96e78c3837b,fe28ed1e-274e-402a-b0c1-3f8d1bf9b7c5,3,True,3
2,5001,fe28ed1e-274e-402a-b0c1-3f8d1bf9b7c5,3,True,3
3,5002,fe28ed1e-274e-402a-b0c1-3f8d1bf9b7c5,3,True,3
4,5003,fe28ed1e-274e-402a-b0c1-3f8d1bf9b7c5,3,True,3
...,...,...,...,...,...
139,6032,bc404235-c139-4efb-90fb-798fbbddc35c,3,True,3
140,dd83c9eb-4a1b-4859-97fb-f2e2603ad690,bc404235-c139-4efb-90fb-798fbbddc35c,3,True,3
141,3003,bc404235-c139-4efb-90fb-798fbbddc35c,3,True,3
142,3004,bc404235-c139-4efb-90fb-798fbbddc35c,3,True,3
